#  <span style="color:darkblue"> Linear Algebraic Systems </span>

<hr style="border:6px solid black"> </hr>

# <span style="color:darkblue"> Learning Objectives </span>

By the end of this module, students will be able to

### *Solve single nonlinear equations*
- Solve a single nonlinear equation by Picard’s method.
- Solve a single nonlinear equation by Newton’s method.
- Determine whether solving a nonlinear equation by fixed-point method will converge.\*
- State the rate of convergence.\*

### *Solve systems of nonlinear equations*
- Formulate residual equations.
- Input a residual of a system of nonlinear equations in a program.
- Solve a system of nonlinear equations by Newton-Raphson.

### *Solve parametric nonlinear systems by continuation methods*
- Solve a parametric nonlinear system using zeroth-order continuation for initial guesses.\*

\*Not covered in tutorial.

<hr style="border:6px solid black"> </hr>

# <span style="color:darkblue">  Solving Single Nonlinear Algebra Equations </span> 

In this section, we will explore Picard’s method and Newton’s method, which are two traditional methods to systematically solve a single nonlinear algebra equation $f(x)=0$.

### Picard's Method
Picard’s method, which is also known as the method of successive substitution, involves simply adding $x$ to both sides of $f(x)=0$. The iterative scheme is

\\[x^{\left(k+1\right)} =f\left(x^{\left(k\right)} \right)+x^{\left(k\right)}.\\]

It is very easy to implement, but it may be very slow to converge.

**Example 1**: Use Picard's method to find the root of function $f(x)=e^{-x}-x$ using initial guess $x^{(0)}=1$. 

We begin by defining a callback functions (f_ex1) to evaluate $f(x)$.

In [2]:
f_ex1(x) = exp(-x) - x

f_ex1 (generic function with 1 method)

We now define a function to perform Picard's method and run it for this problem.

In [3]:
"""
nonlinear_picard

Solves for `f(x) = 0` via Picard iteration where `f` is the residual, `x` is an initial guess, 
`kmax` is the maximum number of iterations, `tol` is the absolute convergence tolerance. This returns
a tuple which holds solution `x` and a boolean value `flag` which is true if the algorithm successfully
converged and false otherwise.
"""
function nonlinear_picard(f,x,kmax,tol)
    f_val = f(x)
    k = 0               
    while abs(f_val) > tol
        x += f_val                                           # Update x
        k += 1                                               # Update iteration counter
        f_val = f(x)                                         # Compute function at next value
        if (k > kmax)
            println("Maximum iterations exceeded.")
            break
        end
    end
    flag = (k > kmax || x == Inf || x == -Inf) ? false : true
    return x, flag
end

x0 = 1.0                     # initial guess
kmax = 20                    # max number of iterations allowed
tol = 1e-5                   # convergence tolerance
x,flag = nonlinear_picard(f_ex1,x0,kmax,tol)
println("The solution is x = $(x)")

The solution is x = 0.5671477142601192


### Newton's Method

Newton’s method is the most common method for solving a single nonlinear equation. The iterative scheme is

\\[x^{(k+1)}=x^{(k)}- \frac{f(x^{(k)})}{f'(x^{(k)})}.\\]

As indicated by the scheme, the derivative $f'(x)$ is required for iterations.

In order to solve **Example 1** using Newton's method, we should derive the derivative term $f'(x)=-e^{-x}-1$.

In [4]:
df_ex1(x) = -exp(-x) - 1

df_ex1 (generic function with 1 method)

We now define a function to perform Newton's method and run it for this problem.

In [18]:
"""
nonlinear_newton

Solves for `f(x) = 0` via Newton's method where `f` is the residual, `df` is the derivative of the residual,
`x` is an initial guess, `kmax` is the maximum number of iterations, and `tol` is the absolute convergence tolerance.
This returns a tuple which holds solution `x` and a boolean value `flag` which is true if the algorithm successfully
converged and false otherwise.
"""
function nonlinear_newton(f,df,x,kmax,tol)
    fval = f(x)
    k = 0
    while abs(fval) > tol
        dfval = df(x)
        x -= fval/dfval
        k += 1
        fval = f(x)
        if k > kmax
            println("Maximum iterations exceeded.")
            break
        end
    end
    flag = (k > kmax || x == Inf || x == -Inf) ? false : true
    return x, flag
end

x0 = 1.0                     # initial guess
kmax = 20                    # max number of iterations allowed
tol = 1e-5                   # convergence tolerance
x,flag = nonlinear_newton(f_ex1,df_ex1,x0,kmax,tol)
println("The solution is x = $(x)")

The solution is x = 0.567143285989123


Netwon's method exhibits quadratic convergence, that is a huge advantage, as the convergence accelerates rapidly near the solution.

<div class="alert alert-block alert-warning">
<b>INTERACTIVE!</b> Run the above snippet of code while varying the initial guess to see how the solution responds. Does this always yield the same value? If so, why? For extreme guesses are any errors encountered? Take a moment and try to reflect on condition(s) under which these may occur.
</div>

# <span style="color:darkblue"> Solving Systems of Nonlinear Algebra Equations </span> 

In this section, we introduce the numerical methods to solve systems of nonlinear algebra equations. The general form is given by:

\\[R_1(x_1,x_2,\ldots,x_n)=0\\
R_2(x_1,x_2,\ldots,x_n)=0\\
\ \ \ \ \ \ \  \vdots\\
R_n(x_1,x_2,\ldots,x_n)=0\\]

We need to be careful to write the equations in a form such that they are equal to zero in order to apply the fixed-point methods. The compact notation of the system of nonlinear equations is $\bf R(\bf x)=\bf0$, with $\bf x$ being the vector of unknowns as $\mathbf x = (x_1,x_2,\ldots,x_n)$, and 

\\[\mathbf{R}=\left\lbrack \begin{array}{c}
R_1 \left(x_1 ,x_2 ,\dots ,x_n \right)\\
R_2 \left(x_1 ,x_2 ,\dots ,x_n \right)\\
\vdots \\
R_n \left(x_1 ,x_2 ,\dots ,x_n \right)
\end{array}\right\rbrack\\]

a vector-valued function which we will refer to as the **residual**.

### Newton-Raphson
A common method for solving nonlinear systems of equations is the Newton–Raphson method. It relies on the same idea as Newton’s method but is now generalized to n dimensions. The iterative scheme is given by

\\[\mathbf{J}(\mathbf{x}^{(k)})(\mathbf{x}^{(k+1)}-\mathbf{x}^{(k)})=-\mathbf{R}(\mathbf{x}^{(k)}),\\]

where the $\mathbf J$ is the Jacobian matrix defined as

\\[\mathbf{J}=\left\lbrack \begin{array}{ccc}
\frac{\partial R_1 }{\partial x_1 } & \dots  & \frac{\partial R_1 }{\partial x_n }\\
\vdots  & \ddots  & \vdots \\
\frac{\partial R_n }{\partial x_1 } & \dots  & \frac{\partial R_n }{\partial x_n }
\end{array}\right\rbrack,\\]

that is to be evaluated at the value $\mathbf{x}^{(k)}$. In component form, the elements of the Jacobian are the partial derivatives $J_{ij}=\frac{\partial R_i}{\partial x_j}$.
It is common to use a shorthand notation $\mathbf{\delta}^{(k+1)}=\mathbf{x}^{(k+1)}-\mathbf{x}^{(k)}$ for the difference between the previous values and the new values of $\mathbf x$. The iterative scheme then becomes the linear system

\\[\mathbf{J}(\mathbf{x}^{(k)})(\mathbf{\delta}^{(k+1)})=-\mathbf{R}(\mathbf{x}^{(k)}),\\]

\\[\mathbf x^{(k+1)}:=\mathbf x^{(k)}+\mathbf{\delta}^{(k+1)}.\\]

We have reduced solving the system of nonlinear equations into an iterative method where, at each step, we need to solve a system of linear equations. Since $\mathbf{x}^{(k)}$ changes at each time step, we need to solve the linear system for different right-hand-side constant vectors and Jacobian matrices.

The implementation of this method goes through the following steps.

1. Create a vector with the residuals.
2. Compute the elements in the Jacobian. 
3. Pick an initial guess $\mathbf{x}^{(0)}$.
4.Iterate until either $\| \mathbf R \|$ and/or $\|\mathbf \delta\|$ are small.

**Example 2:** Use Netwon-Raphson to find the root of the system of equations

\\[f_1(x_1,x_2)=e^{-x_1}-x_2, \\
f_2(x_1,x_2)=x_1+x_2^2-3x_2, \\]

using the initial guess $x_1^{(0)}=0$ and $x_2^{(0)}=0$.

**Solution:**
The residual vector for this problem is

\\[\mathbf{R}=\left\lbrack \begin{array}{c}
e^{-x_1 } -x_2 \\
x_1 +x_2^2 -3x_2 
\end{array}\right\rbrack\\]

If we take the logical choice for the vector of unknowns to be $\mathbf x =(x_1,x_2)$, then the Jacobian is

\\[\mathbf{J}=\left\lbrack \begin{array}{cc}
-e^{-x_1 }  & -1\\
1 & {2x}_2 -3
\end{array}\right\rbrack\\]

We then define corresponding callback functions R_ex2! and J_ex2! which compute the residual and it's jacobian, respectively: 

In [9]:
function R_ex2!(out, x)
    out[1] = exp(-x[1]) - x[2]
    out[2] = x[1] + x[2]^2 -3*x[2]
    return nothing
end

function J_ex2!(out, x)
    out[1,1] = -exp(-x[1])
    out[1,2] = -1
    out[2,1] = 1
    out[2,2] = 2*x[2] - 3
    return nothing
end

J_ex2! (generic function with 1 method)

Then, we implement the iterative algorithm by:

In [11]:
using LinearAlgebra: norm

"""
nonlinear_newton_raphson

Solves for `R(x) = 0` via Newton's method where `R!` is the residual, `J!` is the Jacobian of the residual,
`x` is an initial guess, `kmax` is the maximum number of iterations, and `tol` is the absolute convergence tolerance.
This returns a tuple which holds solution `x` and a boolean value `flag` which is true if the algorithm successfully
converged and false otherwise.
"""
function nonlinear_newton_raphson(R!,J!,x,kmax,tol)
    
    # create intermediate storage
    nx = length(x)                                                
    Rout = zeros(nx)
    Jout = zeros(nx,nx)
    
    R!(Rout, x)
    k = 0
    while norm(Rout) > tol
        J!(Jout, x)                                               # compute Jacobian
        del = -Jout\Rout                                          # calculate step
        x += del                                                  # perform step
        k += 1
        R!(Rout, x)                                               # evaluate residual
        if k > kmax
            println("Maximum iterations exceeded.")
            break
        end
    end
    
    flag = (k > kmax || x == Inf || x == -Inf) ? false : true
    return x, flag
end

x0 = [0.0; 0.0]                                                   # initial guess
kmax = 100                                                        # max number of iterations allowed
tol = 1e-5                                                        # convergence tolerance
x, flag = nonlinear_newton_raphson(R_ex2!,J_ex2!,x0,kmax,tol)
println("The solution is (x1,x2) = ($(x[1]),$(x[2]))")

The solution is (x1,x2) = (0.982752934497163,0.37427931266177783)


# <span style="color:darkblue"> Case Study: Reactor-Separator-Recycle Process for Chlorination of Benzene </span>

In this section, we consider a reactor-separator-recycle process for chlorination of benzene as illustrated in the following figure:

<img src="CSTR_ChloroBenzene.png" width="500">

In the continuous stirred-tank reactor (CSTR), benzene $\text C_6 \text H_6$ (A) reacts with chlorine $\text{Cl}_2$ to yield monochlorobenzene $\text C_6 \text H_5 \text{Cl}$ (B) and dichlorobenzene $\text C_6 \text H_4 \text{Cl}_2$ (C). The chlorine $\text{Cl}_2$ is in excess, thus the reactions can be simplified as:

\\[A\longrightarrow B \\]
\\[B\longrightarrow C \\]

The reaction rates are given by:

\\[r_1=k_1 \frac{y_{3,A}}{y_{3,A} \hat v_A + y_{3,B} \hat v_B + y_{3,C} \hat v_C},\\
r_2 =k_2 \frac{y_{3,B}}{y_{3,A} \hat v_A + y_{3,B} \hat v_B + y_{3,C} \hat v_C}.\\]

The reactor volume is $v=6 \;\text m^3$, the reaction rate constants are $k_1=0.4\;\text h^{-1}$ and $k_2 = 0.055 \;\text h^{-1}$, and the specific volumes of each species are $\hat v_A=8.937\times10^{-2} \;\text m^3/\text{kmol}$, $\hat v_B=1.018\times10^{-1} \;\text m^3/\text{kmol}$, and $\hat v_C=1.130\times10^{-1}\;\text m^3/\text{kmol}$. 

The purporse is to produce $50\; \text{kmol}$ per hour in the outlet stream of the final separator ($F_5=50\; \text{kmol/h}$) and we now want to solve for **the mole fractions of each species** ($y_{3,A}$, $y_{3,B}$, and $y_{3,C}$) in the stream $F_3$.

To solve this problem, we analyze the degrees of freedom for this system:
- Unknowns: $F_1,F_2,F_3,F_4,F_5,F_6,F_7,y_{3,A},y_{3,B},y_{3,C},y_{4,B},y_{4,C},k_1,k_2,\hat v_A, \hat v_B, \hat v_C, v$
- Process specifications: $F_5,k_1,k_2,\hat v_A, \hat v_B, \hat v_C, v$
- Degrees of freedom: 18 unkowns - 7 specifications = 11 degrees of freedom

Thus, 11 independent equations are required for solving the system. Based on the conservation of mass, we can set up the system of equations:

- Mixer:

\\[ F_1+F_7=F_2 \\]

- Reactor:

\\[F_2-r_1v=y_{3,A} F_3\\
0=(r_1-r_2)v+y_{3,B} F_3\\
0=r_2v - y_{3,C}F_3\\]

- Separator 1: 

\\[F_3=F_4+F_7\\
y_{3,B}F_3=y_{4,B}F_4\\
y_{3,C}F_3=y_{4,C}F_4\\]

- Separator 2:

\\[F_4=F_5+F_6\\
y_{4,B}F_4=F_5\\]

- Other Relationships

\\[y_{3,A}+y_{3,B}+y_{3,C}=1\\
y_{4,B}+y_{4,C}=1\\]

Since the compositions are unknown and the reaction rate laws are complex, the system is nonlinear.

### Solutions:
To solve a nonlinear system of equations, we can use the Newton-Raphson method.  The 11 independent equations should be written in residual form by moving every term to one side:

\\[
\begin{array}{l}
F_1 -F_2 +F_7 =0 \\
F_2 - k_1 y_{3,A}v / (y_{3,A} {\hat{v} }_A +y_{3,B} {\hat{v} }_B +y_{3,C} {\hat{v} }_C) - y_{3,A} F_3 =0\\
v(k_2 y_{3,B} -k_1 y_{3,A}) / (y_{3,A} {\hat{v} }_A +y_{3,B} {\hat{v} }_B +y_{3,C} {\hat{v} }_C) + y_{3,B} F_3 =0\\
k_2 y_{3,B}v / (y_{3,A} {\hat{v} }_A +y_{3,B} {\hat{v} }_B +y_{3,C} {\hat{v} }_C)-y_{3,C} F_3 =0 \\
F_3 -F_4 -F_7 =0\\
y_{3,B} F_3 -y_{4,B} F_4 =0\\
y_{3,C} F_3 -y_{4,C} F_4 =0\\
F_4 -F_5 -F_6 =0\\
y_{4,B} F_4 -F_5 =0\\
y_{3,A} +y_{3,B} +y_{3,C} -1=0\\
y_{4,B} +y_{4,C} -1=0
\end{array}
\\]

We define the variable vector for this problem as $\mathbf x=(F_1,F_2,y_{3,A},y_{3,B},y_{3,C},F_3,y_{4,B},y_{4,C},F_4,F_6,F_7)$, then the residual vector is constructed as:

\\[\mathbf{R}\left(\mathbf{x}\right)=\left\lbrack \begin{array}{l}
x_1 -x_2 +x_{11} \\
x_2 -\frac{k_1 x_3 v}{x_3 \hat{v}_A +x_4 \hat{v}_B +x_5 \hat{v}_C }-x_3 x_6 \\
\frac{\left({k_2 x_4 -k}_1 x_3 \right)v}{x_3 \hat{v}_A +x_4 \hat{v}_B +x_5 \hat{v}_C }+x_4 x_6 \\
\frac{k_2 x_4 v}{x_3 \hat{v}_A +x_4 \hat{v}_B +x_5 \hat{v}_C }+x_5 x_6 \\
x_6 -x_9 -x_{11} \\
x_4 x_6 -x_7 x_9 \\
x_5 x_6 -x_8 x_9 \\
x_9 -F_5 -x_{10} \\
x_7 x_9 -F_5 \\
x_3 +x_4 +x_5 -1\\
x_7 +x_8 -1
\end{array}\right\rbrack =\mathbf{0}\\]

Then, we form the Jacobian as:

\\[\mathbf{J}\left(\mathbf{x}\right)=\left\lbrack \begin{array}{ccc}
\frac{\partial R_1 }{\partial x_1 } & \dots  & \frac{\partial R_1 }{\partial x_{11} }\\
\vdots  & \ddots  & \vdots \\
\frac{\partial R_{11} }{\partial x_1 } & \dots  & \frac{\partial R_{11} }{\partial x_{11} }
\end{array}\right\rbrack\\]

Most entries in the Jacobian are zero, the non-zero entries are indicated in the code below. 

<div class="alert alert-block alert-warning">
<b>INTERACTIVE!</b> Fill in the remaining entries to complete the residual and Jacobian below.
</div>

In [20]:
function R_case!(out,x)
    
    # process specifications and constants
    V = 6             # m^3
    F5 = 25           # kmol/h
    Va = 8.937e-2     # m^3/kmol
    Vb = 1.018e-1     # m^3/kmol
    Vc = 1.13e-1      # m^3/kmol
    k1 = 0.4          # h^-1
    k2 = 0.055        # h^-1

    out[1] = x[1] - x[2] + x[11]                          # (eqn 1) x1 - x2 + x11 = 0

    # (eqn 2) x2 - k1*x3*v/(x3*Va + x4*Vb + x5*Vc) - x3*x6 = 0
    out[2] = x[2] - k1*x[3]*V/(x[3]*Va + x[4]*Vb + x[5]*Vc) - x[3]*x[6]

    # (eqn 3) (k2*x4 - k1*x3)*v/(x3*Va + x4*Vb + x5*Vc) + x4*x6 = 0
    out[3] = (k2*x[4] - k1*x[3])*V/(x[3]*Va + x[4]*Vb + x[5]*Vc) + x[4]*x[6]

    # (eqn 4) k2*x4*v/(x3*Va + x4*Vb + x5*Vc) - x5*x6 = 0
    out[4] = k2*x[4]*V/(x[3]*Va + x[4]*Vb + x[5]*Vc) - x[5]*x[6]

    out[5] = x[6] - x[9] - x[11]                          # (eqn 5) x6 - x9 - x11 = 0
    out[6] = x[4]*x[6] - x[7]*x[9]                        # (eqn 6) x4*x6 - x7*x9 = 0
    out[7] = x[5]*x[6] - x[8]*x[9]                        # (eqn 7) x5*x6 - x8*x9 = 0
    out[8] = x[9] - F5 - x[10]                            # (eqn 8) x9 - F5 - x10 = 0
    out[9] = x[7]*x[9] - F5                               # (eqn 9) x7*x9 - F5 = 0
    out[10] = x[3] + x[4] + x[5] - 1                      # (eqn 10) x3 + x4 + x5 - 1 = 0
    out[11] = x[7] + x[8] - 1                             # (eqn 11) x7 + x8 - 1 = 0
    
    return nothing
end

function J_case!(out,x)
    
    # process specifications and constants
    V = 6             # m^3
    F5 = 25           # kmol/h
    Va = 8.937e-2     # m^3/kmol
    Vb = 1.018e-1     # m^3/kmol
    Vc = 1.13e-1      # m^3/kmol
    k1 = 0.4          # h^-1
    k2 = 0.055        # h^-1

    # Set all values in out to zero
    fill!(out, 0.0)
    
    out[1,1] = 1; out[1,2] = -1; out[1,11] = 1 # dR1/dx1, dR1/dx2, and dR1/dx11

    # dR2/dx2, dR2/dx3, dR2/dx4, dR2/dx5, and dR2/dx
    out[2,2] = 1
    out[2,3] = -(k1*V*(x[3]*Va + x[4]*Vb + x[5]*Vc) - k1*x[3]*V*Va)/(x[3]*Va + x[4]*Vb + x[5]*Vc)^2 - x[6]
    out[2,4] = k1*x[3]*V*Vb/(x[3]*Va + x[4]*Vb + x[5]*Vc)^2
    out[2,5] = k1*x[3]*V*Vc/(x[3]*Va + x[4]*Vb + x[5]*Vc)^2
    out[2,6] = -x[3]

      
    # dR3/dx3, dR3/dx4, dR3/dx5, and dR3/dx6
    out[3,3] = (-k1*V*(x[3]*Va + x[4]*Vb + x[5]*Vc) - (k2*x[4] - k1*x[3])*V*Va)/(x[3]*Va + x[4]*Vb + x[5]*Vc)^2
    out[3,4] = (k2*V*(x[3]*Va + x[4]*Vb + x[5]*Vc) - (k2*x[4] - k1*x[3])*V*Vb)/(x[3]*Va + x[4]*Vb + x[5]*Vc)^2 + x[6]
    out[3,5] = -(k2*x[4] - k1*x[3])*V*Vc/(x[3]*Va + x[4]*Vb + x[5]*Vc)^2
    out[3,6] = x[4]
    
    # dR4/dx3, dR4/dx4, dR4/dx5, and dR4/dx6
    out[4,3] = -k2*x[4]*V*Va/(x[3]*Va + x[4]*Vb + x[5]*Vc)^2
    out[4,4] = (k2*V*(x[3]*Va + x[4]*Vb + x[5]*Vc) - k2*x[4]*V*Vb)/(x[3]*Va + x[4]*Vb + x[5]*Vc)^2
    out[4,5] = -k2*x[4]*V*Vc/(x[3]*Va + x[4]*Vb + x[5]*Vc)^2 - x[6]
    out[4,6] = -x[5]

    # dR5/dx6, dR5/dx9, and dR5/dx11
    out[5,6] = 1;  out[5,9] = -1; out[5,11] = -1

    # dR6/dx4, dR6/dx6, dR6/dx7, and dR6/dx9
    out[6,4] = x[6]; out[6,6] = x[4]; out[6,7] = -x[9]; out[6,9] = -x[7]

    # dR7/dx5, dR7/dx6, dR7/dx8, and dR7/dx9
    out[7,5] = x[6]; out[7,6] = x[5]; out[7,8] = -x[9]; out[7,9] = -x[8]

    out[8,9] = 1;    out[8,10] = -1                    # dR8/dx9, dR8/dx10
    out[9,7] = x[9]; out[9,9] = x[7]                   # dR9/dx7, dR9/dx9
    out[10,3] = 1;   out[10,4] = 1;   out[10,5] = 1    # dR10/dx3, dR10/dx4, dR10/dx5
    out[11,7] = 1;   out[11,8] = 1                     # dR11/dx7 and dR11/dx8
                                                                                                                    
    return nothing
end

J_case! (generic function with 1 method)

Then, we implement the Newton-Raphson algorithm by:

In [21]:
x0 = [35.7; 357; 0.9; 0.07; 0.03; 357; 0.7; 0.3; 35.7; 10.7; 321.3]
tol = 1e-4
kmax = 100
x,flag = nonlinear_newton_raphson(R_case!,J_case!,x0,kmax,tol)
println("The mole fraction of each species are")
println("y_{3,A} = $(x[3])")  
println("y_{3,B} = $(x[4])")
println("y_{3,C} = $(x[5])")

The mole fraction of each species are
y_{3,A} = 0.9454833705152079
y_{3,B} = 0.05408780736715945
y_{3,C} = 0.00042882211763266596


<hr style="border:6px solid black"> </hr>

# <span style="color:darkblue">  Question(s) for reflection </span> 

- The Newton-Raphson algorithm makes use of derivative information to compute the direction of steepest descent by solving a linear system of equations. One common issue encountered when applying this approach is potentially overshooting the soluton $\mathbf{x} = \mathbf{x}^*$. How might one solve this issue?